<a href="https://colab.research.google.com/github/HuanAII/MLPforMNIST_classification/blob/main/MLP_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries Pytorch For MNIST Classification**

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets , transforms
from torch.utils.data import DataLoader

# **Load dataset**

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [15]:
train_dataset = datasets.MNIST(train = True , download= True , root= './data' , transform= transform)
test_dataset = datasets.MNIST(train = False , download= True , root= './data' , transform= transform)

In [16]:
# hyperparameter
input_size = 784
hidden_size = 128
output_size = 10
batch_size = 64
learning_rate = 0.1

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# **Define MLP Model**

In [17]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

model = MLP(input_size , hidden_size , output_size)


# **Loss and Optimizer**

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# **Training**

In [21]:
num_epochs = 30
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Flatten the images
        data = data.view(data.size(0), -1)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# ======= Evaluation ========
def check_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data, targets in loader:
            data = data.view(data.size(0), -1)
            scores = model(data)
            _, predictions = scores.max(1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)
    acc = 100 * correct / total
    print(f'Accuracy: {acc:.2f}%')
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Epoch [1/30], Loss: 0.1094
Epoch [2/30], Loss: 0.1900
Epoch [3/30], Loss: 0.0126
Epoch [4/30], Loss: 0.0701
Epoch [5/30], Loss: 0.0478
Epoch [6/30], Loss: 0.0982
Epoch [7/30], Loss: 0.1196
Epoch [8/30], Loss: 0.0159
Epoch [9/30], Loss: 0.0058
Epoch [10/30], Loss: 0.0031
Epoch [11/30], Loss: 0.0051
Epoch [12/30], Loss: 0.0866
Epoch [13/30], Loss: 0.0061
Epoch [14/30], Loss: 0.0027
Epoch [15/30], Loss: 0.0020
Epoch [16/30], Loss: 0.0037
Epoch [17/30], Loss: 0.0016
Epoch [18/30], Loss: 0.0065
Epoch [19/30], Loss: 0.0004
Epoch [20/30], Loss: 0.0004
Epoch [21/30], Loss: 0.0004
Epoch [22/30], Loss: 0.0003
Epoch [23/30], Loss: 0.0025
Epoch [24/30], Loss: 0.0020
Epoch [25/30], Loss: 0.0031
Epoch [26/30], Loss: 0.0009
Epoch [27/30], Loss: 0.0031
Epoch [28/30], Loss: 0.0007
Epoch [29/30], Loss: 0.0005
Epoch [30/30], Loss: 0.0021
Accuracy: 100.00%
Accuracy: 98.18%


In [22]:
torch.save(model.state_dict(), "mlp_mnist.pt")